In [137]:
import tkinter as tk
from tkinter import simpledialog, Toplevel
from tkinter import messagebox
from tkinter import font as tkFont
from PIL import Image, ImageTk
import cv2
import os
from datetime import datetime
import shutil
import matplotlib.pyplot as plt
from tkinter import Tk, Frame, BOTH
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import threading




## 主視窗 
### 基本設定


In [138]:
root = tk.Tk()
root.title("健康紀錄")
root.geometry("400x300")  # 窗口大小


''

Global Variable

In [139]:
name = ""

### 彈出視窗的按鈕的設定

In [140]:
def custom_ask_string(title, prompt):
    dialog = Toplevel(root)
    dialog.title(title)
    dialog.geometry("300x150")  # 設置對話框大小

    label = tk.Label(dialog, text=prompt)
    label.pack(pady=10)

    entry = tk.Entry(dialog, width=30)
    entry.pack(pady=10)

    def on_ok():
        global result
        result = entry.get()
        dialog.destroy()

    def on_cancel():
        global result
        result = None
        dialog.destroy()

    btn_frame = tk.Frame(dialog)
    btn_frame.pack(pady=10)

    btn_ok = tk.Button(btn_frame, text="確定", command=on_ok)
    btn_ok.pack(side=tk.LEFT, padx=5)

    btn_cancel = tk.Button(btn_frame, text="取消", command=on_cancel)
    btn_cancel.pack(side=tk.LEFT, padx=5)

    dialog.transient(root)
    dialog.grab_set()
    root.wait_window(dialog)

    return result

## 主視窗按鈕funciton
### 新建文件

In [141]:
def new_file():
    global name
    # 創建窗口
    new_window = tk.Toplevel(root)
    new_window.title("輸入名字")
    new_window.geometry("300x150")
    
    # 標籤
    label = tk.Label(new_window, text="請輸入名字:")
    label.pack(pady=10)
    
    name_entry = tk.Entry(new_window)
    name_entry.pack(pady=5)
    
    # 按鈕確認
    def confirm():
        global name
        name = name_entry.get()
        new_window.destroy()
    
    confirm_button = tk.Button(new_window, text="確認", command=confirm)
    confirm_button.pack(pady=10)
    pass


In [142]:
import sys
import serial
import time
from PyQt5 import QtWidgets, QtCore
import pyqtgraph as pg
import numpy as np

class RealTimePlot(QtWidgets.QMainWindow):
    def __init__(self, *args, **kwargs):
        super(RealTimePlot, self).__init__(*args, **kwargs)
        self.setWindowTitle('Real-time Data Plot')
        self.setGeometry(100, 100, 1200, 600)

        # Create three plot widgets
        self.plot_widget1 = pg.PlotWidget(title="SPO2")
        self.plot_widget2 = pg.PlotWidget(title="BPM")
        self.plot_widget3 = pg.PlotWidget(title="ECG")

        # Create a layout to hold the plot widgets and the result label and button
        self.layout = QtWidgets.QVBoxLayout()

        # Add plot widgets to the layout
        self.layout.addWidget(self.plot_widget1)
        self.layout.addWidget(self.plot_widget2)

        # Create a label to show the average results
        self.result_label = QtWidgets.QLabel('結果：')
        self.layout.addWidget(self.result_label)

        # Add the plot widget3 to the layout
        self.layout.addWidget(self.plot_widget3)

        # Create a button and add it to the layout
        self.result_button = QtWidgets.QPushButton('察看結果')
        self.layout.addWidget(self.result_button)
        self.result_button.clicked.connect(self.show_results)

        # Create a central widget and set the layout
        self.central_widget = QtWidgets.QWidget()
        self.central_widget.setLayout(self.layout)
        self.setCentralWidget(self.central_widget)

        # Initialize data lists
        self.data1 = []
        self.data2 = []
        self.data3 = []

        # Initialize curves for each plot
        self.curve1 = self.plot_widget1.plot(self.data1, pen='r')
        self.curve2 = self.plot_widget2.plot(self.data2, pen='g')
        self.curve3 = self.plot_widget3.plot(self.data3, pen='b')

        # Initialize serial port
        self.serial_port = serial.Serial('COM5', 9600, timeout=1)

        # Setup timer to update the plots
        self.timer = QtCore.QTimer()
        self.timer.setInterval(50)  # Update interval in milliseconds
        self.timer.timeout.connect(self.update_plot)
        self.timer.start()
         # Variable to store the last recorded data of plot_widget3
        self.recorded_data3 = []

    def update_plot(self):
        if self.serial_port.inWaiting() > 0:
            line = self.serial_port.readline().decode('utf-8').strip()
            try:
                values = line.split('\t')
                if len(values) >= 3:
                    value1 = float(values[0].strip())
                    value2 = float(values[1].strip())
                    value3 = float(values[2].strip())
                    if value1 != 0:
                        self.data1.append(value1)
                    if value2 != 0:
                        self.data2.append(value2)
                    if value3 != 0:
                        self.data3.append(value3)

                    if len(self.data1) > 100:
                        self.data1.pop(0)
                    if len(self.data2) > 100:
                        self.data2.pop(0)
                    if len(self.data3) > 100:
                        self.data3.pop(0)

                    self.curve1.setData(self.data1)
                    self.curve2.setData(self.data2)
                    self.curve3.setData(self.data3)
            except ValueError:
                pass  # Ignore lines that don't contain valid float values

    def show_results(self):
        global name
        if self.data1 and self.data2:
            avg1 = np.mean(self.data1)
            avg2 = np.mean(self.data2)
            #self.result_label.setText(f'結果: SPO2 平均值 = {avg1:.2f}, BPM 平均值 = {avg2:.2f}')
            # color & text
            spo2_text = '血氧過低' if avg1 < 95 else '血氧正常'
            spo2_color = 'red' if avg1 < 95 else 'green'

            bpm_text = '心率偏快' if avg2 > 100 else ('心率偏低' if avg2 < 60 else '心率正常')
            bpm_color = 'red' if (avg2 > 100 or avg2 < 60) else 'green'

            # QLabel
            self.result_label.setText(
                f'<html><body>'
                f'<p style="font-size:20px;">姓名: {name}</p>'
                f'<p style="font-size:20px;">結果:</p>'
                f'<p style="font-size:20px;">SPO2 平均值 = <span style="color:{spo2_color};">{avg1:.2f}</span> - {spo2_text}</p>'
                f'<p style="font-size:20px;">BPM 平均值 = <span style="color:{bpm_color};">{avg2:.2f}</span> - {bpm_text}</p>'
                f'</body></html>'
            )

            # Record current data of plot_widget3
            self.recorded_data3 = self.data3.copy()

            # Clear data3 for plotting the new data
            self.data3.clear()
            self.curve3.setData(self.data3)

        # Stop the timer
        self.timer.stop()

        # Hide the plot widgets and result button
        self.plot_widget1.hide()
        self.plot_widget2.hide()
        self.plot_widget3.hide()
        self.result_button.hide()

        # Show recorded data3 in plot_widget3
        self.plot_widget3.show()
        self.curve3.setData(self.recorded_data3)

        # Resize window to fit only the result label and plot_widget3
        self.resize(1200, 400)


### 開始測量
需撰寫 儲存資料 這個步驟

In [143]:
def measure_data():
    def run_qt():
        app = QtWidgets.QApplication(sys.argv)
        main = RealTimePlot()
        main.show()
        sys.exit(app.exec_())

    threading.Thread(target=run_qt).start()

### 查詢資料
需要再加入 讀取資料夾(日期) 裡面的 
1. 血氧
2. 溫度
3. 脈搏
並將資料作折線圖


In [144]:

def view_file():
    pass

### 退出

In [145]:
def exit_app():
    root.destroy()

### 主視窗 圖片

In [146]:
def load_image(path, max_size):
    img = Image.open(path)
    img.thumbnail(max_size, Image.LANCZOS)  # 使用 Image.LANCZOS 替代 Image.ANTIALIAS
    return ImageTk.PhotoImage(img)


In [147]:
image_path = "./health_no1.png"  # 确保路径正确
max_size = (200, 200)  # 设置缩放后的最大尺寸
image = load_image(image_path, max_size)

# 创建一个Label小部件来显示图片，并保存图像引用
image_label = tk.Label(root, image=image)
image_label.image = image  # 這行代碼保存了圖像的引用
image_label.place(x=50, y=50)  # 调整位置使图片在窗口中居中显示

TclError: image "pyimage12" doesn't exist

### 主視窗 按鈕本體 字體與位置

In [ ]:
# 設置字體 與 大小
button_font = tkFont.Font(family="TkDefaultFont", size=12)

btn_new = tk.Button(root, text="輸入名字", command=new_file, font=button_font)
btn_new.place(x=280, y=80)

btn_new = tk.Button(root, text="開始測量", command=measure_data, font=button_font)
btn_new.place(x=280, y=120)

btn_exit = tk.Button(root, text="退出", command=exit_app, font=button_font)
btn_exit.place(x=280, y=200)


### mainloop


In [ ]:
root.mainloop()
